## Making Primary imports

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow.keras.applications import VGG16

import os


## Creating the Model

In [2]:
vgg16_obj = VGG16(include_top = False, input_shape = (224,224,3))    #  include_top = False is used to skip the layer from flattern
for layer in vgg16_obj.layers:             # Off the training of the trainable parameters
    layer.trainable = False
vgg16_obj.summary()

2022-09-15 16:57:54.482079: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-15 16:57:54.631813: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-15 16:57:54.632744: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-15 16:57:54.634679: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

58900480/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
______________________________________________________________

In [4]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense,Flatten
f1 = Flatten()(vgg16_obj.output)
final_layer = Dense(58, activation='softmax')(f1)
final_layer

<KerasTensor: shape=(None, 58) dtype=float32 (created by layer 'dense')>

In [5]:
model = Model(inputs=vgg16_obj.input,outputs=final_layer)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

## Creating the augmented dataset

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
traffic_datagen = ImageDataGenerator(rescale=1/255,
                                  shear_range=0.7,
                                  zoom_range=0.5)

In [10]:
path='../input/traffic-sign-dataset-classification/traffic_Data/DATA'
traffic_data =traffic_datagen.flow_from_directory(
    directory=path,
    target_size=(224,224),
    batch_size=3,
    class_mode="categorical",
    )

Found 4170 images belonging to 58 classes.


## Training the model

In [11]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])   

In [12]:
model.fit_generator(traffic_data, epochs=10)

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2022-09-15 17:03:26.733665: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10


2022-09-15 17:03:28.351116: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


1390/1390 [==============================] - 79s 52ms/step - loss: 1.5151 - accuracy: 0.7374
Epoch 2/10
1390/1390 [==============================] - 62s 45ms/step - loss: 0.4605 - accuracy: 0.9137
Epoch 3/10
1390/1390 [==============================] - 63s 45ms/step - loss: 0.2683 - accuracy: 0.9465
Epoch 4/10
1390/1390 [==============================] - 63s 45ms/step - loss: 0.1861 - accuracy: 0.9631
Epoch 5/10
1390/1390 [==============================] - 63s 45ms/step - loss: 0.1379 - accuracy: 0.9724
Epoch 6/10
1390/1390 [==============================] - 62s 45ms/step - loss: 0.1231 - accuracy: 0.9779
Epoch 7/10
1390/1390 [==============================] - 60s 43ms/step - loss: 0.0859 - accuracy: 0.9803
Epoch 8/10
1390/1390 [==============================] - 60s 43ms/step - loss: 0.0966 - accuracy: 0.9835
Epoch 9/10
1390/1390 [==============================] - 60s 43ms/step - loss: 0.0695 - accuracy: 0.9847
Epoch 10/10
1390/1390 [==============================] - 60s 43ms/step - lo

In [13]:
model.save('Traffic_sign_prediction.h5')